In [1]:
from IPython.core.display import display, HTML
display(HTML("""
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
"""))

In [2]:
import pandas 
import numpy 
import os
from os import listdir
import ipywidgets as widgets
from ipywidgets import interact
from plotly import tools
import plotly.graph_objs as graph_objects
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import Javascript
init_notebook_mode(connected=True)

In [3]:
class Layout(graph_objects.Layout):
    def __init__(self, title, x_title, y_title, width = 800, height = 400):
        super().__init__(title = title, 
                         xaxis = dict(title = x_title,
                                showgrid = False, 
                                linecolor = 'black',
                                ticks = 'outside',
                                titlefont = dict(size = 20)), 
                         yaxis = dict(title = y_title,
                                showgrid = True, 
                                linecolor = 'black',
                                ticks = 'outside'),
                         margin = dict(t = 50),
                         autosize = False,
                         width = width,
                         height = height)

class ForgePlot():
    trace_list = []
    chart_meta = None
    x_axis_title = "Not Set"
    y_axis_title = "Not Set"
    y_axis2_title = "Not Set"
    chart_title = "Unset Title"
    y_axis_options = {}
    lower_time_limit = '2017-11-07 17:00:00'
    upper_time_limit = '2017-11-07 18:00:00'
    full_data = None
    time_range = dict()
    
    @classmethod
    def build_data(cls):
        filename_list = os.listdir('data')
        First = True
        for file_name in filename_list:
            print(file_name)
            filename, extension = os.path.splitext(file_name)
            if extension == '.csv':
                data = pandas.read_csv("%s/data/%s" % (os.getcwd(), file_name), index_col = 0)
            elif extension == '.txt':
                data = pandas.read_csv("%s/data/%s" % (os.getcwd(), file_name), delimiter = '\t', index_col = 0)
            data.index = pandas.to_datetime(data.index)
            #print(data)
            if First:
                First = False
                cls.full_data = data
            else:
                cls.full_data = cls.full_data.join(data)
            
        new_columns = []
        for column in cls.full_data.columns:
            new_col = column.strip()
            new_columns.append(new_col)
            #print('>%s<' % new_col)
        cls.full_data.columns = new_columns
        time_range = cls.get_time_range()
         
        
    @classmethod
    def set_meta(cls):
        cls.chart_meta = cls.full_data.loc[cls.lower_time_limit:cls.upper_time_limit]
    
    
    @classmethod
    def add_trace(cls, key, name):
        cls.trace_list.append(graph_objects.Scatter(
            x = cls.chart_meta.index,
            y = cls.chart_meta[key],
            name = name,
            mode = 'lines',
        ))
    
    @classmethod
    def add_mass_list(cls, mass_list):
        for mass in mass_list:
            cls.add_mass_trace(mass)
    
    @classmethod
    def add_mass_trace(cls, mass):
        cls.trace_list.append(graph_objects.Scatter(
        x = cls.chart_meta.index,
        y = cls.chart_meta[mass],
        connectgaps = True,
        name = mass,
        mode = 'lines',
        yaxis = 'y2',
        #visible = 'legendonly'
    ))
    
    @classmethod
    def set_axis_names(cls, x_axis_title, y_axis_title):
        cls.x_axis_title = x_axis_title
        cls.y_axis_title = y_axis_title
    
    @classmethod
    def set_chart_title(cls, title):
        cls.chart_title = title
    
    @classmethod
    def set_yaxis2_option(cls, options):
        cls.y_axis_options = options
        
    @classmethod
    def build_chart(cls, chart_title, x_axis_title, y_axis_title):
        layout = Layout(chart_title, x_axis_title, y_axis_title)
        if cls.y_axis_options != {}:
            layout['yaxis2'] = cls.y_axis_options
        layout['legend'] = {'x': 1.15}
        figure = graph_objects.Figure(data = cls.trace_list, layout = layout)
        iplot(figure)
        
    @classmethod
    def get_time_range(cls):
        if cls.full_data is not None:
            result = dict()
            result['lower'] = cls.full_data.index[0]
            result['upper'] = cls.full_data.index[len(ForgePlot.full_data.index)-1]
            return result
        
    @classmethod
    def get_trace_list_keys(cls):
        keys = []
        for trace in cls.trace_list:
            keys.append(trace.name)
        return keys
    
    @classmethod
    def set_yaxis2(cls, title):
       cls.set_yaxis2_option({'title': title, 'side': 'right', 'overlaying':'y', 'exponentformat':'power'})

In [4]:
print(os.getcwd())
display(widgets.HTML(
    value=("<b>Instructions</b><br>Place the files you wish to plot in the data folder that is the same directory as the notebook. The click 'View' to verify the files are in the correct location")
))

def view_files(sender):
    file_list = os.listdir('data')
    text_string = ""
    for filename in file_list:
        display(widgets.Label(filename))
    

view_files_button = widgets.Button(
    description='View Files',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='View The Forge Nano Files',
    icon='check'
)

view_files_button.on_click(view_files)
file_listview = widgets.Label('No Files!')
display(view_files_button, file_listview)
%pwd


C:\Users\ALyon\start


HTML(value="<b>Instructions</b><br>Place the files you wish to plot in the data folder that is the same direct…

Button(button_style='success', description='View Files', icon='check', style=ButtonStyle(), tooltip='View The …

Label(value='No Files!')

'C:\\Users\\ALyon\\start'

In [5]:
def build_chart_build_button_build():
    def build_chart(sender):
        fplot.build_chart(fplot.chart_title, fplot.x_axis_title, fplot.y_axis_title)
    
    build_chart_button = widgets.Button(
        description='Build Chart',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Build a Sexy Chart',
        icon='check'
    )

    build_chart_button.on_click(build_chart)
    
    display(build_chart_button)
    
    
def build_trace_accordion():
    reg_col_list = []
    mass_col_list = []

    for column in fplot.full_data:
        if 'Mass ' in column:
            mass_col_list.append(column)
        else:
            reg_col_list.append(column)

    reg_col_selection = widgets.Dropdown(
        options = reg_col_list,
        description = 'Select Trace',)

    mass_col_selection = widgets.Dropdown(
        options = mass_col_list,
        description = 'Select Mass Trace',)

    reg_col_name_entry = widgets.Text('Trace Name')

    
    def add_regular_trace(sender):
        choice = reg_col_selection.value

        trace_name = reg_col_name_entry.value
        fplot.add_trace(choice, trace_name)
        print('Added: %s' % trace_name)
        print('Existing Traces by Name')
        print('=======================')
        [print(name) for name in fplot.get_trace_list_keys()]
        if len(fplot.get_trace_list_keys()) == 1:
            build_chart_build_button_build()
        

    def add_mass_trace(sender):
        choice = mass_col_selection.value
        fplot.add_mass_trace(choice)
        print('Added Mass: %s' % choice)
        print('Existing Traces by Name')
        print('=======================')
        [print(name) for name in fplot.get_trace_list_keys()]
        if len(fplot.get_trace_list_keys()) == 1:
            build_chart_build_button_build()
        
    reg_col_add_button = widgets.Button(
        description='Add Trace',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Add Trace',
        icon='check'
    )

    reg_col_add_button.on_click(add_regular_trace)

    reg_entry_group = widgets.HBox([widgets.Label('Trace Name'), reg_col_name_entry, reg_col_add_button])

    reg_add_group = widgets.VBox([reg_col_selection, reg_entry_group])

    mass_col_add_button = widgets.Button(
        description='Add Mass Trace',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Add Mass Trace',
        icon='check'
    )

    mass_col_add_button.on_click(add_mass_trace)

    mass_entry_group = widgets.HBox([mass_col_add_button])

    mass_add_group = widgets.VBox([mass_col_selection, mass_entry_group])

    accordion_trace = widgets.Accordion(children=[reg_add_group, mass_add_group])
    accordion_trace.set_title(0, 'Regular Trace')
    accordion_trace.set_title(1, 'Mass Trace')
    display(accordion_trace)
    print('Existing Traces by Name')
    print('=======================')
    [print(name) for name in fplot.get_trace_list_keys()]
    
    
def build_accordion():
    chart_title_entry = widgets.Text()
    chart_title_entry.value = 'Basic Chart'
    title_group = widgets.HBox([widgets.Label(value="Chart Title"), chart_title_entry])

    xaxis_entry = widgets.Text()
    xaxis_entry.value = 'X Axis'
    xaxis_group = widgets.HBox([widgets.Label(value="X-Axis Title"), xaxis_entry])

    yaxis_entry = widgets.Text()
    yaxis_entry.value = 'Y Axis'
    yaxis_group = widgets.HBox([widgets.Label(value="Y-Axis Title"), yaxis_entry])

    yaxis2_entry = widgets.Text()
    yaxis2_entry.value = 'Y Axis 2'
    yaxis2_group = widgets.HBox([widgets.Label(value="Y-Axis 2 Title"), yaxis2_entry])

    time_range_slider = widgets.IntRangeSlider(
        value=[0, len(fplot.full_data)-1],
        min=0,
        max=len(fplot.full_data)-1,
        step=1,
        description='Time Range',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
    )

    time_range_slider = widgets.IntRangeSlider(
        value=[0, len(fplot.full_data)-1],
        min=0,
        max=len(fplot.full_data)-1,
        step=1,
        description='Time Range',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=False
    )

    readback_low = widgets.Label(str(fplot.full_data.index[0]))
    readback_buffer = widgets.Label('  <----->  ')
    readback_high = widgets.Label(str(fplot.full_data.index[len(fplot.full_data.index)-1]))
    readback_group = widgets.HBox([readback_low, readback_buffer, readback_high])

    def on_value_change(change):
        low = change['new'][0]
        high = change['new'][1]
        readback_low.value = str(fplot.full_data.index[low])
        readback_high.value = str(fplot.full_data.index[high])

    time_range_slider.observe(on_value_change, names='value')

    range_group = widgets.VBox([time_range_slider, readback_group])

    def set_chart_basics(sender):
        fplot.chart_title = chart_title_entry.value
        fplot.x_axis_title = xaxis_entry.value
        fplot.y_axis_title = yaxis_entry.value
        fplot.set_yaxis2(yaxis2_entry.value)
        fplot.lower_time_limit = readback_low.value
        fplot.upper_time_limit = readback_high.value
        fplot.set_meta()
        build_trace_accordion()


    chart_basics_button = widgets.Button(
        description='Set!',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Set Values',
        icon='check'
    )

    chart_basics_button.on_click(set_chart_basics)

    accordion_basic = widgets.Accordion(children=[title_group, xaxis_group, 
                                                  yaxis_group, yaxis2_group,
                                                 range_group])
    accordion_basic.set_title(0, 'Chart Title')
    accordion_basic.set_title(1, 'X-Axis')
    accordion_basic.set_title(2, 'Y-Axis')
    accordion_basic.set_title(3, 'Y-Axis 2')
    accordion_basic.set_title(4, 'Time Range')


    display(accordion_basic, chart_basics_button)
    


In [6]:
fplot = ForgePlot
def build_data(sender):
    fplot.build_data()
    build_accordion()
    
file_select_button = widgets.Button(
    description='Import Data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Import this very important data',
    icon='check'
)

file_select_button.on_click(build_data)

display(file_select_button)

Button(button_style='success', description='Import Data', icon='check', style=ButtonStyle(), tooltip='Import t…

171107 - FN0127p1-1 - 52g Umicore - 1C NiO.csv
171107 - FN0127p1-1 - 52g Umicore - 1C NiO.txt


Accordion(children=(HBox(children=(Label(value='Chart Title'), Text(value='Basic Chart'))), HBox(children=(Lab…

Button(button_style='success', description='Set!', icon='check', style=ButtonStyle(), tooltip='Set Values')

Accordion(children=(VBox(children=(Dropdown(description='Select Trace', options=('mfc_101_pv', 'mfc_102_pv', '…

Existing Traces by Name
Added: TI 102
Existing Traces by Name
TI 102


Button(button_style='success', description='Build Chart', icon='check', style=ButtonStyle(), tooltip='Build a …

Added Mass: Mass 32
Existing Traces by Name
TI 102
Mass 32
